In [8]:
from abc import *
from dataclasses import dataclass
import random
import sys

class StoreGuideline(metaclass=ABCMeta):
  @abstractmethod
  def add_customer(cls, new_customer): #new_customer는 Person
    pass
  
  @abstractmethod
  def save_point(cls, num, total_buy):
    pass

  @abstractmethod
  def calc_basket(self, basket): # basket은 [(상품명, 수량),]
    pass

  @abstractmethod
  def pick_items(self): 
    pass
# -------------abstract class--------------
class MyException(Exception):
    pass

@dataclass
class Product:
  name: str
  price: int

class Store(StoreGuideline):
  customers = {}
  mandatory_stock = [Product('water', 500), Product('ramen', 1000), Product('coffee', 800) ]
  discount_dict = {}
  asset = [0]
  lottery_list = []

  def __init__(self, store_name, stocks = []):
    self.store_name = store_name
    stocks += Store.mandatory_stock
    self.stock_dict = {product.name: [product.price, 10] for product in stocks}

  def check_asset(self):
    print(Store.asset[0])

  def add_stock(self, *args):
    for bundle in args:
      product, quantity = bundle[0], bundle[1]
      self.stock_dict[product.name] = [product.price, quantity]
      print(f'{product.name}을 각 {quantity}개만큼 재고에 추가하였습니다')
    print(f'현 시점 재고목록:', end= ' ')
    self.print_stock()
  
  def print_stock(self):
    print([f'{name}:{self.stock_dict[name][1]}' for name in self.stock_dict.keys()])
  
  @classmethod
  def add_customer(cls, new_customer):
    def first_join(cls, newbie):
        new_id = new_customer.phone_num[-4:]
        if new_id not in cls.customers.keys():
            cls.customers[new_id] = [newbie.name, 500]
            print(f'최초 가입 이벤트! {newbie.name}님 500점 적립되셨습니다.')
    return first_join(cls, new_customer)
  
  @classmethod
  def save_point(cls, num, total_buy):
    cls.customers[num][1] += int(total_buy//10)
    print(f'{cls.customers[num][0]}님의 새로 적립된 포인트: {total_buy//10} 점\n총 가용 포인트: {cls.customers[num][1]}점')
  
  @classmethod
  def use_point(cls, num, total_buy):
    while cls.customers[num][1] > 0:
      print(f'{cls.customers[num][0]} 님의 총 가용 포인트는 {cls.customers[num][1]} 점입니다')
      total_buy -= cls.customers[num][1]
      cls.customers[num][1] = 0
      print(f'포인트를 사용하여 {total_buy}원 결제 도와드리겠습니다')
      cls.fortune[0] += total_buy
      return total_buy
  
  @classmethod
  def alter_sale(cls, product_name, rate): # 수동으로 세일품목 추가 및 삭제
    if rate == 0:
      del cls.discount_dict[product_name]
    else:
      cls.discount_dict[product_name] = rate
    print(f'할인 이벤트 수정 완료 - 현재 할인품목: {cls.discount_dict}')
    
  @classmethod
  def payback(cls, num, *args):
        for bundle in args:
            reusable, quantity = bundle[0], bundle[1]
            if reusable == "milk_carton":
                cls.customers[num][1] += quantity * 5
                print(f'{cls.customers[num][0]}님, 포인트가 {quantity*5}만큼 적립되었습니다. 총 포인트 : {cls.customers[num][1]}')
            elif reusable == "empty_bottle":
                cls.customers[num][1] += quantity * 10
                print(f'{cls.customers[num][0]}님, 포인트가 {quantity*10}만큼 적립되었습니다. 총 포인트 : {cls.customers[num][1]}')
            else:
                raise MyException("그 물건은 페이백이 안 됩니다..")
                
  @classmethod 
  def lottery(cls, entry_num):
    cls.lottery_list.append(entry_num)
    print('현재 응모 상황 : ', cls.lottery_list)
    
    if len(cls.lottery_list) == 3:
        lotteries = random.sample(cls.lottery_list, 2)
        def printing(lst:list):
            for lottery in lst:
                yield lottery
        for i in printing(lotteries):
            cls.customers[i][1] += 100
            print(f'당첨자 {cls.customers[i][0]}({i})님께 포인트가 100만큼 적립됩니다. 총 포인트 {cls.customers[i][1]}')
        cls.lottery_list = []

  def calc_basket(self, basket):
    total = 0
    for bundle in basket:
      if bundle[0] in Store.discount_dict.keys():
        print(f'{bundle[0]}는 {Store.discount_dict[bundle[0]] *100 }% 행사 상품입니다')
        discounted_price = self.stock_dict[bundle[0]][0] * (1 - Store.discount_dict[bundle[0]])
        total += discounted_price * bundle[1]
      else:
        total += self.stock_dict[bundle[0]][0] * bundle[1]
    print(f'구매하시는 제품의 총 금액은 {total} 원입니다')
    return total


  def pick_items(self, **kargs):
    print('편의점의 제품 목록:', end= '')
    print([f'{key}({value[0]}원) {value[1]}개' for key, value in self.stock_dict.items()])
    basket = []
    if kargs['product'] in self.stock_dict.keys() and kargs['num'] <= self.stock_dict[kargs['product']][1]:
      basket.append((kargs['product'], kargs['num']))
      return basket
        
  
  def update_stock(self, sold_items):
    for bundle in sold_items:
      self.stock_dict[bundle[0]][1] -= bundle[1]
      if self.stock_dict[bundle[0]][1] == 0:
        del self.stock_dict[bundle[0]]
  

class Person(Store):
    """
    편의점 클래스를 부모로 하는 사람 함수.
    이름, 전화번호를 초기 설정으로 받는다.
    보유금액은 기본적으로 1000 ~ 10000원 사이의 금액으로 설정되며 사용자가 기입할 수 있다.
    각 Person 클래스마다 개별 장바구니 변수를 설정해놓고 장바구니 카트에 아이템을 담을 수 있게 해놓았다.
    각 함수는 의사코드 참조.
    """

    def __init__(self, name, phone_num, money = (random.randrange(1, 10))*1000, basket = {}):
        self.name = name
        self.phone_num = phone_num
        self.money = money
        self.basket = basket

  def check_money(self):
    print(f'나의 현재 통장 잔고: {self.money}')
  
  def work(self, hour):
    self.money += 10000 * hour
    print(f'{hour} 시간을 노동하여 돈을 벌었습니다.')
    self.check_money

  def buy_product(self, store, **kargs):
    customer_id = self.phone_num[-4:]
    basket = store.pick_items(**kargs)
    if not basket: 
      print('아무것도 구매를 원하지 않으시므로 종료합니다')
      sys.exit()
    basket_total = store.calc_basket(basket)
    super().save_point(customer_id, basket_total)    
    final_price = super().use_point(customer_id, basket_total)
    assert self.money >= final_price, '잔액 부족'
    print(f'{self.money} 원 받았습니다')
    self.money -= final_price 
    print(f'{self.money} 원 거슬러 드렸습니다. 이용해 주셔서 감사합니다.')
    store.update_stock(basket)
    

  def check_point(self):
    id = self.phone_num[-4:]
    print(f'현재 {self.name}의 멤버십 포인트는 {super().customers[id][1]}점입니다.')

In [9]:
# Test Code
# ------------------ basic option---------------------
cola = Product('cola', 1000)
sandwich = Product('sandwich', 2000)
milk = Product('milk', 1200)

cu = Store('씨유테스트점')
cu.add_stock((cola,3), (sandwich,5), (milk, 6))

cu.print_stock()
Store.alter_sale('water', 0.1)
print("\n")
# ----------------------------------------------------
# -----------------Person option----------------------
A = Person('Sally', '01000001111')
B = Person('Bob', '01000002222')
C = Person('Taylor', '01000003333')
D = Person('Jacky', '01000004444')
cu.add_customer(A)
cu.add_customer(A) 
# 최초가입이벤트(first_join) 문구는 한 번만 나오는 것을 확인
cu.add_customer(B)
cu.add_customer(C)
cu.add_customer(D)
print(cu.customers)
print("\n")
# ------------------------------------------------------
# --------------Payback Event---------------------------
A.check_point()
cu.payback('1111', ('milk_carton', 5), ('empty_bottle', 7))
# cu.payback('1111', ('water_bottle', 1))
# 우유갑과 공병을 제외한 다른 상품이 들어오면 사용자 예외(MyException) 발생
print("\n")
# ------------------------------------------------------
# -----------------Lottery Event------------------------
cu.lottery('1111')
cu.lottery('2222')
cu.lottery('3333')
cu.lottery('4444')
# ------------------------------------------------------

cola을 각 3개만큼 재고에 추가하였습니다
sandwich을 각 5개만큼 재고에 추가하였습니다
milk을 각 6개만큼 재고에 추가하였습니다
현 시점 재고목록: ['water:10', 'ramen:10', 'coffee:10', 'cola:3', 'sandwich:5', 'milk:6']
['water:10', 'ramen:10', 'coffee:10', 'cola:3', 'sandwich:5', 'milk:6']
할인 이벤트 수정 완료 - 현재 할인품목: {'water': 0.1}


최초 가입 이벤트! Sally님 500점 적립되셨습니다.
최초 가입 이벤트! Bob님 500점 적립되셨습니다.
최초 가입 이벤트! Taylor님 500점 적립되셨습니다.
최초 가입 이벤트! Jacky님 500점 적립되셨습니다.
{'1111': ['Sally', 500], '2222': ['Bob', 500], '3333': ['Taylor', 500], '4444': ['Jacky', 500]}


현재 Sally의 멤버십 포인트는 500점입니다.
Sally님, 포인트가 25만큼 적립되었습니다. 총 포인트 : 525
Sally님, 포인트가 70만큼 적립되었습니다. 총 포인트 : 595


현재 응모 상황 :  ['1111']
현재 응모 상황 :  ['1111', '2222']
현재 응모 상황 :  ['1111', '2222', '3333']
당첨자 Sally(1111)님께 포인트가 100만큼 적립됩니다. 총 포인트 695
당첨자 Bob(2222)님께 포인트가 100만큼 적립됩니다. 총 포인트 600
현재 응모 상황 :  ['4444']
